In [1]:
import pandas as pd
import numpy as np
from scipy.stats import f_oneway
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import statsmodels.api as sa
import statsmodels.formula.api as sfa
import scikit_posthocs as sp

## Organizando os dados

In [2]:
relation = pd.DataFrame([{'tratamento': 'RT + Milho', 'sigla': 'A'},
            {'tratamento': 'RT + Milho + Aditivo', 'sigla': 'B'},
            {'tratamento': 'RT + Soja', 'sigla': 'C'},
            {'tratamento': 'RT + Soja + Aditivo', 'sigla': 'D'}])

In [3]:
relation

,tratamento,sigla
0,RT + Milho,A
1,RT + Milho + Aditivo,B
2,RT + Soja,C
3,RT + Soja + Aditivo,D


In [78]:
# dataframe com dados do exercício
df = pd.DataFrame([{'tratamento': 'A', 'rep-1': 20, 'rep-2': 22, 'rep-3': 23, 'rep-4': 21, 'tipo': 'milho', 'aditivo': 'não-aditivo'}, 
                   {'tratamento': 'B', 'rep-1': 24, 'rep-2': 25, 'rep-3': 23, 'rep-4': 26, 'tipo': 'milho', 'aditivo': 'aditivo'},
                   {'tratamento': 'C', 'rep-1': 27, 'rep-2': 27, 'rep-3': 25, 'rep-4': 29, 'tipo': 'soja', 'aditivo': 'não-aditivo'},
                   {'tratamento': 'D', 'rep-1': 26, 'rep-2': 28, 'rep-3': 30, 'rep-4': 32, 'tipo': 'soja', 'aditivo': 'aditivo'}]) 

In [79]:
df

,tratamento,rep-1,rep-2,rep-3,rep-4,tipo,aditivo
0,A,20,22,23,21,milho,não-aditivo
1,B,24,25,23,26,milho,aditivo
2,C,27,27,25,29,soja,não-aditivo
3,D,26,28,30,32,soja,aditivo


## Letra C 

In [80]:
df_anova = df.drop(columns=['tipo', 'aditivo']).set_index('tratamento').T.reset_index().drop(columns=['index'])

In [81]:
df_anova

tratamento,A,B,C,D
0,20,24,27,26
1,22,25,27,28
2,23,23,25,30
3,21,26,29,32


In [82]:
# anova aplicado á todas repetições de cada tratamento
from scipy.stats import f_oneway
f_oneway(df_anova['A'], df_anova['B'], df_anova['C'], df_anova['D'])

F_onewayResult(statistic=13.263157894736842, pvalue=0.0004062083471110179)

In [85]:
df_teste = pd.concat([df[[classificacao, 'rep-1']].rename(columns={'rep-1':valor}), 
                      df[[classificacao, 'rep-2']].rename(columns={'rep-2':valor}), 
                      df[[classificacao, 'rep-3']].rename(columns={'rep-3':valor}), 
                      df[[classificacao, 'rep-4']].rename(columns={'rep-4':valor})])

In [86]:
df_teste

,tratamento,valor
0,A,20
1,B,24
2,C,27
3,D,26
0,A,22
1,B,25
2,C,27
3,D,28
0,A,23
1,B,23


In [101]:
lm = sfa.ols('valor ~ C(tratamento)', data=df_teste).fit()
anova = sa.stats.anova_lm(lm)

In [102]:
anova

,df,sum_sq,mean_sq,F,PR(>F)
C(tratamento),3.0,126.0,42.000000,13.263158,0.000406
Residual,12.0,38.0,3.166667,NaN,NaN


## Letra E

In [84]:
classificacao = 'tratamento'
valor = 'valor'

In [87]:
tukey = sp.posthoc_tukey(df_teste, group_col=classificacao, val_col=valor)

In [88]:
tukey

,A,B,C,D
A,1.000000,0.133300,0.004361,0.001000
B,0.133300,1.000000,0.245876,0.017321
C,0.004361,0.245876,1.000000,0.421238
D,0.001000,0.017321,0.421238,1.000000


In [107]:
tukey_pair = pairwise_tukeyhsd(endog=df_teste[valor],
                          groups=df_teste[classificacao],
                          alpha=0.05)

print(tukey_pair)

Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
     A      B      3.0 0.1333 -0.7362  6.7362  False
     A      C      5.5 0.0044  1.7638  9.2362   True
     A      D      7.5  0.001  3.7638 11.2362   True
     B      C      2.5 0.2459 -1.2362  6.2362  False
     B      D      4.5 0.0173  0.7638  8.2362   True
     C      D      2.0 0.4212 -1.7362  5.7362  False
----------------------------------------------------


## Letra F

In [109]:
classificacao = 'tipo'
valor = 'valor'

In [117]:
relation = pd.DataFrame([{'tratamento': 'milho', 'sigla': 'A'},
            {'tratamento': 'milho', 'sigla': 'B'},
            {'tratamento': 'soja', 'sigla': 'C'},
            {'tratamento': 'soja', 'sigla': 'D'}])

In [118]:
relation

,tratamento,sigla
0,milho,A
1,milho,B
2,soja,C
3,soja,D


In [110]:
df_teste = pd.concat([df[[classificacao, 'rep-1']].rename(columns={'rep-1':valor}), 
                      df[[classificacao, 'rep-2']].rename(columns={'rep-2':valor}), 
                      df[[classificacao, 'rep-3']].rename(columns={'rep-3':valor}), 
                      df[[classificacao, 'rep-4']].rename(columns={'rep-4':valor})])

In [116]:
df_teste

,tipo,valor
0,milho,20
1,milho,24
2,soja,27
3,soja,26
0,milho,22
1,milho,25
2,soja,27
3,soja,28
0,milho,23
1,milho,23


In [129]:
def pairwise_scheffe(df, treatment, value):
    p_adj = sp.posthoc_scheffe(df, group_col=treatment, val_col=value)
    mean = df.groupby(treatment).mean()
    
    combinations = list(itertools.combinations(p_adj.columns, 2))
    
    result = []
    for combination in combinations: 
        first = mean.loc[combination[0], value]
        second = mean.loc[combination[1], value]
        p_value = p_adj.loc[combination[0], combination[1]]
            
        result.append([combination[0], combination[1], abs(first-second), p_value])
    
    result_scheffe = pd.DataFrame(result, columns=['group1', 'group2', 'meandiff', 'p-value'])
    
    return result_scheffe

In [130]:
scheffe = sp.posthoc_scheffe(df_teste, group_col=classificacao, val_col=valor)

In [131]:
scheffe

,não-aditivo,aditivo
não-aditivo,1.000000,0.134876
aditivo,0.134876,1.000000


In [132]:
scheffe_pair = pairwise_scheffe(df_teste, classificacao, valor)

scheffe_pair

,group1,group2,meandiff,p-value
0,não-aditivo,aditivo,2.5,0.134876


## Letra G

In [119]:
classificacao = 'aditivo'
valor = 'valor'

In [120]:
df_teste = pd.concat([df[[classificacao, 'rep-1']].rename(columns={'rep-1':valor}), 
                      df[[classificacao, 'rep-2']].rename(columns={'rep-2':valor}), 
                      df[[classificacao, 'rep-3']].rename(columns={'rep-3':valor}), 
                      df[[classificacao, 'rep-4']].rename(columns={'rep-4':valor})])

In [121]:
df_teste

,aditivo,valor
0,não-aditivo,20
1,aditivo,24
2,não-aditivo,27
3,aditivo,26
0,não-aditivo,22
1,aditivo,25
2,não-aditivo,27
3,aditivo,28
0,não-aditivo,23
1,aditivo,23


In [127]:
scheffe = sp.posthoc_scheffe(df_teste, group_col=classificacao, val_col=valor, p_adjust=0.5)

In [133]:
scheffe

,não-aditivo,aditivo
não-aditivo,1.000000,0.134876
aditivo,0.134876,1.000000


In [134]:
scheffe_pair = pairwise_scheffe(df_teste, classificacao, valor)

scheffe_pair

,group1,group2,meandiff,p-value
0,não-aditivo,aditivo,2.5,0.134876
